# Go for a Walk and Arrive at the Answer: Reasoning Over Paths in Knowledge Bases using Reinforcement Learning

Rajarshi Das, Shehzaad Dhuliawala, Manzil Zaheer, Luke Vilnis, Ishan Durugkar, Akshay Krishnamurthy, Alex Smola, Andrew McCallum (ICLR 2018)

* This paper presents a method for efficiently searching the graph for answer-providing paths using reinforcement learning (RL) conditioned on the input question, eliminating any need for pre-computed paths.
* In this formulated RL problem, the goal is to take an optimal sequence of decisions (choices of relation edges) to maximize the expected reward (reaching the correct answer node). We call the RL agent **MINERVA for "Meandering In Networks of Entities to Reach Verisimilar Answers."**

Properties of MINERVA:
1. built-in flexibility to take paths of variable length
2. needs no pretraining and trains on the knowledge graph from scratch with reinforcement learning; no other supervision or fine-tuning is required
3. computationally efficient, since by searching in a small neighborhood around the query entity it avoids ranking all entities in the KB
4. the reasoning paths found by our agent automatically form an interpretable record for its predictions

## MDP FORMULATION

1. **Environment**
    
    The Knowledge Base (KB) is a collection of facts stored as triplets $(e_1, r, e_2)$ where $e_1, e_2 \in \mathcal{E}$ are entities linked by relation $r \in \mathcal{R}$. The KB can be represented as a knowledge graph $\mathcal{G} = (\mathit{V},\mathit{E},\mathcal{R})$ where $\mathit{V}=\mathcal{E}$, $\mathit{E}$ is the set of directed edges defined by the facts in the KB. For each edge $(e_1, r, e_2) \in \mathit{E}$, the authors add an edge $(e_2, r^{-1}, e_1)$ where $r^{-1}$ is the inverse relation. This is important because it actually equips the agent to undo a potentially wrong decision as it can retract back to the current node in the next step.
    
    * States: The state space consists of all valid combinations of the quad $(e_t, e_{1q}, r_q, e_{2q})$ where $e_t$ is the location of the agent at time t, and $e_{2q}$ is the answer to the query $(e_{1q}, r_q, ?)$
    * Observations: Given the state $s = (e_t, e_{1q}, r_q, e_{2q})$, the observation of the agent is $(e_t, e_{1q}, r_q)$
    * Actions: An agent at each state has option to select which outgoing edge it wishes to take having the knowledge of the label of the edge r and destination vertex v, i.e. $\mathcal{A}_\mathcal{S} = \{(e_t, r, v) \in \mathit{E} : \mathcal{S} = (e_t,e_{1q},r_q,e_{2q}), r \in \mathcal{R},v \in \mathit{V}\} \cup \{(s,\phi,s)\}$. The last action is a special action called **NO_OP** which goes from a node to itself. Some questions are easier to answer and needs lesser steps of reasoning than others. This design decision allows the agent to remain at a node for any number of time steps. This is especially helpful when questions are easier to answer and needs lesser steps of reasoning than others. This design decision allows the agent to remain at a node for any number of time steps. This is especially helpful when the agent has managed to reach a correct answer at a time step t < T and can continue to stay at the 'answer node' for the rest of the time steps. Alternatively, we could have allowed the agent to take a special 'STOP' action, but we found the current setup to work sufficiently well.
    * Transitions: The *environment evolves deterministically*, i.e. the agent traverses the edge chosen with probability 1
    * Rewards: Agent gets a reward of +1 if the final node is the answer node, otherwise it gets 0 reward.
    
    The resulting problem is a ***finite horizon, deterministic, partially observed*** MDP

2. **Policy**

    The authors aim to create a randomized history-dependent policy $\pi$. An agent based on LSTM encodes the history $H_t$ as a continuous vector $\mathbf{h}_t \in \mathbb{R}^{2d}$. We also have embedding matrix $\mathbf{r} \in \mathbb{R}^{|\mathcal{R}|\times d}$ and $\mathbf{e} \in \mathbb{R}^{|\mathcal{E}|\times d}$ for the binary relations and entities respectively. The history embedding for $H_t = (H_{t-1},A_{t-1},O_t)$ (where $O_t = (e_t, e_{1q}, r_q)$ ) is updated according to LSTM dynamics: $$\mathbf{h}_t = LSTM(\mathbf{h}_{t-1},[\mathbf{r}[\mathbf{a}_{t-1},:];\mathbf{e}[\mathbf{e}_t,:]])$$ Based on the history embedding $\mathbf{h}_t$, the policy network makes the decision to choose an action from all available actions $(\mathcal{A}_{\mathcal{S}_t})$ conditioned on the query relation. In order to do this, first construct the embedding matrix for the available actions, i.e. for all actions $A = (e_t, r_l, e_d) \in \mathcal{A}_{e_t}$, $$\mathbf{A}_t = stack(concatenate(\mathbf{r}[r_l,:],\mathbf{e}[e_d,:]))$$ Given this embedding, the agent behavior is given by: $$\mathbf{d}_t = softmax(\mathbf{A}_t (\mathbf{W}_2 ReLU(\mathbf{W_1}[\mathbf{h}_t;\mathbf{e}[e_t,:];\mathbf{r}[r_q,:]])))$$ $$A_t \sim Categorical(\mathbf{d}_t)$$ The procedure of building $\mathbf{A}_t$ is invariant to the order in which edges are presented (as desired) and falls in purview of neural networks designed to be [permutation invariant](https://papers.nips.cc/paper/6931-deep-sets.pdf).
    
3. **Training**

TODO: Study [REINFORCE](https://link.springer.com/article/10.1007/BF00992696)